# CRUD

> TEMP! Eventually have admin panel with auth to make this easier? This isn't to be exported as a .py file as this is only run in notebooks

Use `Shift + s` to stop the Jupyter forms running

> Currently it only supports **markdown from gists**, which Solveit supports exporting as

In [ ]:
#| default_exp crud

In [ ]:
from dialoghelper import *

In [ ]:
# fc_tool_info()

Tools available from `fastcore.tools`:

- &`rg`: Run the `rg` command with the args in `argstr` (no need to backslash escape)
- &`sed`: Run the `sed` command with the args in `argstr` (e.g for reading a section of a file)
- &`view`: View directory or file contents with optional line range and numbers
- &`create`: Creates a new file with the given content at the specified path
- &`insert`: Insert new_str at specified line number
- &`str_replace`: Replace first occurrence of old_str with new_str in file
- &`strs_replace`: Replace for each str pair in old_strs,new_strs
- &`replace_lines`: Replace lines in file using start and end line-numbers

In [ ]:
from fastcore.tools import *

In [ ]:
from casblog.core import Post, db, PROJECT_ROOT
import os
import httpx
import re
from datetime import datetime
from fastlite import *
import json

In [ ]:
def fetch_gist_markdown(gist_url: str) -> tuple[str, str]:
    """Fetch markdown content and filename from a gist URL."""
    match = re.search(r'gist\.github\.com/[^/]+/([a-f0-9]+)', gist_url)
    if not match: raise ValueError(f"Invalid gist URL: {gist_url}")
    
    resp = httpx.get(f'https://api.github.com/gists/{match.group(1)}')
    resp.raise_for_status()
    
    md_file = next((f for f in resp.json()['files'].values() 
                    if f['filename'].endswith('.md')), None)
    if not md_file: raise ValueError("No markdown file found in gist")
    
    return md_file['content'], md_file['filename']

In [ ]:
def get_db(prod: bool = False):
    """Get database connection."""
    return database(PROJECT_ROOT / "data" / ("prod.db" if prod else "dev.db"))

In [ ]:
db = get_db(prod=False)
assert db.tables, "Database should have tables"

## Add Post

In [ ]:
def save_post(db, post: Post) -> Post:
    """Save a Post to the given database, returning new Post with id."""
    result = db.t.post.insert(post)
    return Post(**{**post.__dict__, 'id': result['id']})

In [ ]:
def parse_categories(s: str) -> str:
    """Convert comma-separated string to JSON array string."""
    if not s.strip(): return '[]'
    return json.dumps([c.strip() for c in s.split(',') if c.strip()])

In [ ]:
post_gist = "https://gist.github.com/stantonius/96fd3f0693c8566a8d7ef6d6dff325bf"

In [ ]:
#| notest
content, filename = fetch_gist_markdown(post_gist)

try:
    post = Post(
        title=input(f"Title [{filename}]: ").strip() or filename.replace('.md', ''),
        content=content,
        slug=input("Slug (or Enter to auto-generate): ").strip() or None,
        categories=parse_categories(input("Categories (comma-separated): "))
    )
except KeyboardInterrupt:
    print("\nCancelled.")


saved = save_post(get_db(prod=True), post)

## Delete Post

In [ ]:
#| notest
try:
    # Fetch all posts from prod
    prod_db = get_db(prod=True)
    posts = list(prod_db.t.post.rows)

    # Display posts
    print("Posts in prod database:\n")
    for p in posts:
        print(f"  {p['id']}: {p['title']}")

    # Get selection
    ids_str = input("\nEnter post IDs to delete (comma-separated), or Enter to cancel: ").strip()
    if not ids_str:
        print("Cancelled.")
    else:
        ids = [int(i.strip()) for i in ids_str.split(',')]
        
        # Show what will be deleted
        print(f"\n⚠️  About to delete {len(ids)} post(s):")
        for pid in ids:
            p = next((p for p in posts if p['id'] == pid), None)
            if p: print(f"  - {p['title']}")
        
        # Confirm
        confirm = input("\nType 'DELETE' to confirm: ").strip()
        if confirm == 'DELETE':
            for pid in ids:
                prod_db.t.post.delete(pid)
                print(f"🗑️  Deleted post id={pid}")
            print("\n✅ Done!")
        else:
            print("Cancelled.")
except KeyboardInterrupt:
    print("\nCancelled.")